https://medium.datadriveninvestor.com/how-to-use-python-to-calculate-intrinsic-value-in-15-minutes-8c76e2ca2d8a

In [17]:
import pandas as pd
import requests
import pickle

In [18]:
symbol = 'AAPL'

pickle_file = f'../data/{symbol}.pkl'

In [19]:
def get_and_pickle_data(symbol: str):

    url = "https://yh-finance.p.rapidapi.com/stock/v2/get-financials"

    querystring = {"symbol": symbol, "region": 'US'}

    headers = {
        "X-RapidAPI-Key": "6e24ec4876msha70babfb873be61p173927jsn48df65d05e0c",
        "X-RapidAPI-Host": "yh-finance.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    df = response.json()

    df1 = pd.json_normalize(df)

    with open(f'../data/{symbol}.pkl', 'wb') as file:
        pickle.dump(df1, file)

In [20]:
get_and_pickle_data(symbol)

,symbol,loading,errorList,financialsTemplate.code,financialsTemplate.maxAge,cashflowStatementHistory.cashflowStatements,cashflowStatementHistory.maxAge,balanceSheetHistoryQuarterly.balanceSheetStatements,balanceSheetHistoryQuarterly.maxAge,earnings.maxAge,...,timeSeries.trailingSellingGeneralAndAdministration,timeSeries.trailingBasicAverageShares,timeSeries.trailingDilutedEPS,timeSeries.trailingDilutedAverageShares,timeSeries.trailingBasicEPS,timeSeries.timestamp,meta.symbol,meta.start,meta.end,meta.timeUnit
0,AAPL,False,[],N,1,"[{'investments': {'raw': -9560000000, 'fmt': '...",86400,"[{'totalLiab': {'raw': 290020000000, 'fmt': '2...",86400,86400,...,"[{'dataId': 20159, 'asOfDate': '2022-12-31', '...",[],[],[],[],"[1569801600, 1601424000, 1632960000, 1664496000]",AAPL,493590046,1678024329,annual


In [37]:
def intrinsic_value(symbol_df, compound_rate, discount_rate, terms):
    
    df1 = symbol_df

    cashflow = df1.loc[:, df1.columns.str.contains('cashflowStatementHistoryQuarterly.cashflowStatements')]
    incomestatement = df1.loc[:, df1.columns.str.contains('incomeStatementHistory.incomeStatementHistory')]

    def clean(x, y):
        df = pd.DataFrame(x.iloc[0, 0])
        df = df.loc[:, df.columns.str.contains(y)]
        df = pd.json_normalize(df[y])
        df = pd.DataFrame(df.raw)
        df.columns = [y]
        return (df)

    ebit = clean(incomestatement, 'incomeBeforeTax')
    depreciation = clean(cashflow, 'depreciation')
    capex = clean(cashflow, 'capitalExpenditures')

    df = pd.concat([ebit, depreciation, capex], axis=1)

    mean_capex = df['capitalExpenditures'].mean()

    display(df)

    return

    df1 = df.loc[0:1]
    df1['Average Capex'] = mean_capex
    df1['Owners Earnings'] = df1['incomeBeforeTax'] + \
        df1['depreciation']-df1['Average Capex']

    return
    def compound(x, y):
        z = (1+x)**y
        return (z)

    def discount(x, y):
        z = 1/(1+x)**y
        return (z)

    dfc = []
    for y in range(1, terms+1):
        z = compound(compound_rate, y)
        dfc.append(z)

    dfc1 = pd.DataFrame(dfc)

    dfd = []
    for y in range(1, terms+1):
        z = discount(discount_rate, y)
        dfd.append(z)

    dfd1 = pd.DataFrame(dfd)

    calcs_table = pd.concat([dfc1, dfd1], axis=1)
    calcs_table.columns = ["Compound", "Discount"]

    calcs_table["Amounts"] = calcs_table['Compound']*calcs_table['Discount']

    PV_multiplier = pd.DataFrame(
        calcs_table['Amounts'].tail(1)).reset_index(drop=True)
    PV_multiplier.columns = ["PV_multiplier"]
    PV_multiplier = PV_multiplier['PV_multiplier']/discount_rate

    calcs_table1 = calcs_table.append(calcs_table.sum().rename('Total'))
    DCF_multiplier = pd.DataFrame(
        calcs_table1['Amounts'].tail(1)).reset_index(drop=True)
    DCF_multiplier.columns = ["DCF_multiplier"]

    final_df = pd.concat([df1, PV_multiplier, DCF_multiplier], axis=1)

    final_df['OE*PV'] = final_df['Owners Earnings']*final_df['PV_multiplier']
    final_df['OE*DCF'] = final_df['Owners Earnings']*final_df['DCF_multiplier']
    final_df['Intrinsic Value'] = final_df['OE*PV'] + final_df['OE*DCF']

    # df = response.json()
    # df1 = pd.json_normalize(df)
    # display(df1)
    return

    df2 = pd.DataFrame(df1)

    df3 = df2.loc[:, df1.columns.str.contains(
        'timeSeries.annualDilutedAverageShares')]
    df4 = clean(df3, 'reportedValue')
    shares = df4[::-1].reset_index(drop=True)
    shares.columns = ["Shares Outstanding"]

    final_df = pd.concat([final_df, shares], axis=1)
    final_df['Per Share'] = final_df['Intrinsic Value'] / \
        final_df['Shares Outstanding']
    final_df = final_df.iloc[0:1]

    return (final_df)

with open(pickle_file, 'rb') as file:
    symbol_df = pickle.load(file)

intrinsic_value(symbol_df, compound_rate=0.1, discount_rate=0.05, terms=5)


,incomeBeforeTax,depreciation,capitalExpenditures
0,119103000000,2916000000,-3787000000
1,109207000000,2865000000,-3289000000
2,67091000000,2805000000,-2102000000
3,65737000000,2737000000,-2514000000
